In [215]:
from gurobipy import *
import math

In [216]:
model = Model("matchup-schedualing")

In [217]:
I = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
P = list(range(1,len(I)))
K = list(range(1,9))

In [218]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
df = pd.read_excel('Boxoffice.xlsx')

In [219]:
print(df)

          j=1  j=2  j=3  j=4  j=5  j=6  j=7  j=8  j=9  j=10  j=11  j=12  j=13  \
i=1,K=1     0    1    2   16    5   11   17    8   12     9     3    19    14   
i=2,K=1    19    0    9   19    2   18   11    3    5    17     7    20    17   
i=3,K=1     5   11    0   18   20   18    6    5    8    15    13    18    12   
i=4,K=1     7   16   13    0    7    6    3   13   20     7     1    19    20   
i=5,K=1    19   13   17   16    0    7    4    7    6    11    18     2    12   
i=6,K=1     6    5   10    8    6    0    9    8   16     1    16    13     9   
i=7,K=1    15   15   19    8    6   16    0   19   14     9     5    13    11   
i=8,K=1     2   14   12   19   13   14    5    0    9    14    18     6    16   
i=9,K=1     8   18   14    3    2   16   19    4    0    16    18     2     1   
i=10,K=1   15    9    7   12    8   15   17    2    2     0    10     5    11   
i=11,K=1    1   19   20   10   12    9   18   16   18    11     0     3    17   
i=12,K=1    4   17   11    5

In [220]:
import numpy as np
B = np.zeros((8, 16, 16), dtype = np.int)

In [221]:
for i in range(0,16):
    for j in range(0,16):
        for k in range(0,8):
            B[k,i,j] = df.iat[(16*k + i), j]

In [222]:
model.update

<bound method Model.update of <gurobi.Model Continuous instance matchup-schedualing: 0 constrs, 0 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>>

In [223]:
u = {}
for i in I:
    for p in P:
        u[i,p] = model.addVar(vtype = "B", name = "u(%s%s)"%(i,p))
v = {}
for k in K:
    for p in P:
        v[k,p] = model.addVar(vtype = "B", name = "v(%s%s)"%(k,p))
w = {}
for j in I:
    for i in range(j + 1, len(I) + 1):
        for k in K:
            for p in P:
                w[i,j,k,p] = model.addVar(vtype = "B", name = "w(%s%s%s%s)"%(i,j,k,p))
model.update()

In [224]:
for j in I:
    for i in range(j + 1, len(I) + 1):
        for p in P:
            model.addConstr(0.5*(u[i,p] + u[j,p]) + (10**15)*(u[i,p]-1) + (10**15)*(u[j,p]-1) <= u[i,p]*u[j,p])
            model.addConstr(0.5*(u[i,p] + u[j,p]) >= u[i,p]*u[j,p])

In [225]:
for j in I:
    for i in range(j + 1, len(I) + 1):
        for p in {15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2}:
            model.addConstr(u[i,p]*u[j,p] <= u[i,math.floor(p/2)])
            model.addConstr(u[i,p]*u[j,p] <= 1 - u[j,math.floor(p/2)])

In [226]:
for i in I:
    model.addConstr(quicksum(u[i,p] for p in range(math.floor(len(I)/2), len(I))) == 1)

In [227]:
for p in P:
    model.addConstr(quicksum(u[i,p] for i in I) == 2)

In [228]:
for k in K:
    model.addConstr(quicksum(v[k,p] for p in range(math.floor(len(I)/2), len(I))) <= 1)
for k in K:
    model.addConstr(quicksum(v[k,p] for p in range(1, math.floor(len(I)/2))) <= 1)
for p in P:
    model.addConstr(quicksum(v[k,p] for k in K) == 1)

In [229]:
for j in I:
    for i in range(j + 1, len(I) + 1):
        for k in K:
            for p in P:
                model.addConstr(w[i,j,k,p] <= u[i,p]*u[j,p])
                model.addConstr(w[i,j,k,p] <= v[k,p])
                model.addConstr(w[i,j,k,p] >= u[i,p]*u[j,p] + v[k,p]-1)
                model.addConstr(w[i,j,k,p] >= 0)

In [230]:
obj = quicksum(quicksum(quicksum(quicksum(B[(k-1),(i-1),(j-1)]*w[i,j,k,p] 
                                          for p in P) for k in K) for i in range(j + 1, len(I) + 1)) for j in I)

model.setObjective(obj, GRB.MAXIMIZE)

In [ ]:
model.optimize()

Optimize a model with 28862 rows, 14760 columns and 43808 nonzeros
Model has 35760 quadratic constraints
Variable types: 0 continuous, 14760 integer (14760 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [5e-01, 1e+15]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
  QRHS range       [1e+00, 2e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 14400 rows and 0 columns
Presolve time: 2.75s
Presolved: 14462 rows, 14760 columns, 29408 nonzeros
Variable types: 0 continuous, 14760 integer (14760 binary)

Root relaxation: objective 1.935000e+04, 20067 iterations, 0.72 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 19284.3750    0 6898          - 19284.3750      -    

  1559  1199 16853.1250   55 14556  259.00000 16853.1250  6407%  2701 4444s
  1594  1226 16851.3750   56 14555  259.00000 16851.3750  6406%  2757 4603s
  1629  1245 16722.8750   57 14452  259.00000 16722.8750  6357%  2837 4766s
  1667  1266 16611.7500   58 14353  259.00000 16611.7500  6314%  2903 4959s
  1707  1289 16609.5000   59 14352  259.00000 16609.5000  6313%  2993 5138s
  1765  1328 11245.4940   60 9728  259.00000 16607.7500  6312%  3038 5327s
  1819  1363 12385.1250   61 10794  259.00000 16247.3605  6173%  3109 5512s
  1875  1395 14625.0750   62 12666  259.00000 16111.9916  6121%  3151 5716s
  1937  1438 14274.0536   63 12358  259.00000 16111.9916  6121%  3204 5932s
  2001  1473 14180.8795   64 12285  259.00000 16111.9916  6121%  3252 6140s
  2086  1531 12522.6172   65 10907  259.00000 16111.9916  6121%  3273 6352s
  2203  1582 12682.0000   66 11031  259.00000 16111.9916  6121%  3264 6557s
  2331  1650 12480.5885   68 10865  259.00000 16111.9916  6121%  3241 6746s
  2492  1713 

 25557 19848 2441.46495  215 10568  286.00000 2818.40076   885%  1879 55855s
 25558 19849 2818.27964   55 10605  286.00000 2818.27964   885%  1878 55866s
 25559 19850 2018.25060  101 10619  286.00000 2818.23217   885%  1878 55878s
 25560 19850 2818.20054   45 10602  286.00000 2818.20054   885%  1878 55889s
 25561 19851 2818.11243   56 10574  286.00000 2818.11243   885%  1878 55905s
 25562 19852 1276.32667   81 10609  286.00000 2818.07909   885%  1878 55918s
 25563 19852 2815.32613   84 10415  286.00000 2815.32613   884%  1878 55980s
 25564 19853  897.12719  134 10433  286.00000 2814.34630   884%  1878 56000s
 25565 19854  393.50000  114 10556  286.00000 2813.83491   884%  1878 56014s
 25566 19854 2012.12351  101 10533  286.00000 2813.64656   884%  1878 56027s
 25567 19855 2811.10800   85 10435  286.00000 2811.10800   883%  1878 56073s
 25568 19856 2810.46611   87 10459  286.00000 2810.46611   883%  1878 56091s
 25569 19856 2810.04613   76 10497  286.00000 2810.04613   883%  1878 56119s

 26000 20105 2415.28283   49 8100  286.00000 2499.95360   774%  2009 61417s
 26027 20118 2370.36310   51 7381  286.00000 2499.95360   774%  2013 61513s
 26061 20140 2333.82264   53 7176  286.00000 2499.95360   774%  2015 61611s
 26096 20160 2275.18371   56 6696  286.00000 2499.95360   774%  2018 61705s
 26146 20193 2061.71363   58 5661  286.00000 2499.95360   774%  2020 61791s
 26206 20219 2030.88593   59 5410  286.00000 2499.95360   774%  2018 61953s
 26253 20238 1879.07858   62 4899  286.00000 2499.95360   774%  2022 62042s
 26293 20246 1527.21628   64 3681  286.00000 2499.95360   774%  2024 62174s
 26330 20263 1144.53539   72 2181  286.00000 2499.95360   774%  2028 62343s
 26393 20289  421.70594   96  392  286.00000 2470.07666   764%  2030 62506s
 26450 20311 2436.36462   49 7708  286.00000 2470.07666   764%  2034 62667s
 26497 20329 2320.75328   52 6899  286.00000 2470.07666   764%  2038 62863s
 26530 20347 2217.91691   54 6194  286.00000 2470.07666   764%  2045 62990s
 26587 20371

In [180]:
for i in model.getVars():
    print(i.varname, i.x)

u(11) 0.0
u(12) 0.0
u(13) 0.0
u(14) 1.0
u(15) 0.0
u(16) 0.0
u(17) 0.0
u(21) 0.0
u(22) 0.0
u(23) 0.0
u(24) 0.0
u(25) 0.0
u(26) 1.0
u(27) 0.0
u(31) 0.0
u(32) 0.0
u(33) 0.0
u(34) 0.0
u(35) 0.0
u(36) 0.0
u(37) 1.0
u(41) 0.0
u(42) 0.0
u(43) 0.0
u(44) 0.0
u(45) 1.0
u(46) 0.0
u(47) 0.0
u(51) 0.0
u(52) 0.0
u(53) 1.0
u(54) 0.0
u(55) 0.0
u(56) 0.0
u(57) 1.0
u(61) 0.0
u(62) 1.0
u(63) 0.0
u(64) 1.0
u(65) 0.0
u(66) 0.0
u(67) 0.0
u(71) 1.0
u(72) 0.0
u(73) 1.0
u(74) 0.0
u(75) 0.0
u(76) 1.0
u(77) 0.0
u(81) 1.0
u(82) 1.0
u(83) 0.0
u(84) 0.0
u(85) 1.0
u(86) 0.0
u(87) 0.0
v(11) 0.0
v(12) 0.0
v(13) 0.0
v(14) 0.0
v(15) 0.0
v(16) 0.0
v(17) 0.0
v(21) 1.0
v(22) 0.0
v(23) 0.0
v(24) 0.0
v(25) 0.0
v(26) 0.0
v(27) 1.0
v(31) 0.0
v(32) 0.0
v(33) 0.0
v(34) 1.0
v(35) 0.0
v(36) 0.0
v(37) 0.0
v(41) 0.0
v(42) 0.0
v(43) 0.0
v(44) 0.0
v(45) 0.0
v(46) 0.0
v(47) 0.0
v(51) 0.0
v(52) 1.0
v(53) 0.0
v(54) 0.0
v(55) 1.0
v(56) 0.0
v(57) 0.0
v(61) 0.0
v(62) 0.0
v(63) 1.0
v(64) 0.0
v(65) 0.0
v(66) 1.0
v(67) 0.0
w(2111) 0.0
w(2112) 

w(7257) 0.0
w(7261) 0.0
w(7262) 0.0
w(7263) 0.0
w(7264) 0.0
w(7265) 0.0
w(7266) 1.0
w(7267) 0.0
w(8211) 0.0
w(8212) 0.0
w(8213) 0.0
w(8214) 0.0
w(8215) 0.0
w(8216) 0.0
w(8217) 0.0
w(8221) 0.0
w(8222) 0.0
w(8223) 0.0
w(8224) 0.0
w(8225) 0.0
w(8226) 0.0
w(8227) 0.0
w(8231) 0.0
w(8232) 0.0
w(8233) 0.0
w(8234) 0.0
w(8235) 0.0
w(8236) 0.0
w(8237) 0.0
w(8241) 0.0
w(8242) 0.0
w(8243) 0.0
w(8244) 0.0
w(8245) 0.0
w(8246) 0.0
w(8247) 0.0
w(8251) 0.0
w(8252) 0.0
w(8253) 0.0
w(8254) 0.0
w(8255) 0.0
w(8256) 0.0
w(8257) 0.0
w(8261) 0.0
w(8262) 0.0
w(8263) 0.0
w(8264) 0.0
w(8265) 0.0
w(8266) 0.0
w(8267) 0.0
w(9211) 0.0
w(9212) 0.0
w(9213) 0.0
w(9214) 0.0
w(9215) 0.0
w(9216) 0.0
w(9217) 0.0
w(9221) 0.0
w(9222) 0.0
w(9223) 0.0
w(9224) 0.0
w(9225) 0.0
w(9226) 0.0
w(9227) 0.0
w(9231) 0.0
w(9232) 0.0
w(9233) 0.0
w(9234) 0.0
w(9235) 0.0
w(9236) 0.0
w(9237) 0.0
w(9241) 0.0
w(9242) 0.0
w(9243) 0.0
w(9244) 0.0
w(9245) 0.0
w(9246) 0.0
w(9247) 0.0
w(9251) 0.0
w(9252) 0.0
w(9253) 0.0
w(9254) 0.0
w(9255) 0.0
w(92

w(8553) 0.0
w(8554) 0.0
w(8555) 0.0
w(8556) 0.0
w(8557) 0.0
w(8561) 0.0
w(8562) 0.0
w(8563) 0.0
w(8564) 0.0
w(8565) 0.0
w(8566) 0.0
w(8567) 0.0
w(9511) 0.0
w(9512) 0.0
w(9513) 0.0
w(9514) 0.0
w(9515) 0.0
w(9516) 0.0
w(9517) 0.0
w(9521) 0.0
w(9522) 0.0
w(9523) 0.0
w(9524) 0.0
w(9525) 0.0
w(9526) 0.0
w(9527) 0.0
w(9531) 0.0
w(9532) 0.0
w(9533) 0.0
w(9534) 0.0
w(9535) 0.0
w(9536) 0.0
w(9537) 0.0
w(9541) 0.0
w(9542) 0.0
w(9543) 0.0
w(9544) 0.0
w(9545) 0.0
w(9546) 0.0
w(9547) 0.0
w(9551) 0.0
w(9552) 0.0
w(9553) 0.0
w(9554) 0.0
w(9555) 0.0
w(9556) 0.0
w(9557) 0.0
w(9561) 0.0
w(9562) 0.0
w(9563) 0.0
w(9564) 0.0
w(9565) 0.0
w(9566) 0.0
w(9567) 0.0
w(10511) 0.0
w(10512) 0.0
w(10513) 0.0
w(10514) 0.0
w(10515) 0.0
w(10516) 0.0
w(10517) 0.0
w(10521) 0.0
w(10522) 0.0
w(10523) 0.0
w(10524) 0.0
w(10525) 0.0
w(10526) 0.0
w(10527) 0.0
w(10531) 0.0
w(10532) 0.0
w(10533) 0.0
w(10534) 0.0
w(10535) 0.0
w(10536) 0.0
w(10537) 0.0
w(10541) 0.0
w(10542) 0.0
w(10543) 0.0
w(10544) 0.0
w(10545) 0.0
w(10546) 0.0
w

w(13742) 0.0
w(13743) 0.0
w(13744) 0.0
w(13745) 0.0
w(13746) 0.0
w(13747) 0.0
w(13751) 0.0
w(13752) 0.0
w(13753) 0.0
w(13754) 0.0
w(13755) 0.0
w(13756) 0.0
w(13757) 0.0
w(13761) 0.0
w(13762) 0.0
w(13763) 0.0
w(13764) 0.0
w(13765) 0.0
w(13766) 0.0
w(13767) 0.0
w(14711) 0.0
w(14712) 0.0
w(14713) 0.0
w(14714) 0.0
w(14715) 0.0
w(14716) 0.0
w(14717) 0.0
w(14721) 0.0
w(14722) 0.0
w(14723) 0.0
w(14724) 0.0
w(14725) 0.0
w(14726) 0.0
w(14727) 0.0
w(14731) 0.0
w(14732) 0.0
w(14733) 0.0
w(14734) 0.0
w(14735) 0.0
w(14736) 0.0
w(14737) 0.0
w(14741) 0.0
w(14742) 0.0
w(14743) 0.0
w(14744) 0.0
w(14745) 0.0
w(14746) 0.0
w(14747) 0.0
w(14751) 0.0
w(14752) 0.0
w(14753) 0.0
w(14754) 0.0
w(14755) 0.0
w(14756) 0.0
w(14757) 0.0
w(14761) 0.0
w(14762) 0.0
w(14763) 0.0
w(14764) 0.0
w(14765) 0.0
w(14766) 0.0
w(14767) 0.0
w(15711) 0.0
w(15712) 0.0
w(15713) 0.0
w(15714) 0.0
w(15715) 0.0
w(15716) 0.0
w(15717) 0.0
w(15721) 0.0
w(15722) 0.0
w(15723) 0.0
w(15724) 0.0
w(15725) 0.0
w(15726) 0.0
w(15727) 0.0
w(15731) 0.0